In [ ]:
import cracknuts as cn

In [ ]:
cracker = cn.new_cracker('192.168.0.19')
# cracker

In [ ]:
import random
import time
from enum import Enum

class AlgType(Enum):  # Algrithom type
    AES_ENC = 1
    AES_DEC = 2
    DES_ENC = 3
    DES_DEC = 4

sample_length = 1024 * 20
# alg = AlgType.AES_ENC # 修改这个值改变算法
alg = AlgType.DES_ENC # 修改这个值改变算法

# AES Command
cmd_set_aes_enc_key = "01 00 00 00 00 00 00 10"
cmd_set_aes_dec_key = "01 01 00 00 00 00 00 10"
cmd_aes_enc = "01 02 00 00 00 00 00 10"
cmd_aes_dec = "01 03 00 00 00 00 00 10"
key_aes = "11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff"
aes_data_len = 16

# DES Command
cmd_set_des_enc_key = "02 00 00 00 00 00 00 08"
cmd_set_des_dec_key = "02 01 00 00 00 00 00 08"
cmd_des_enc = "02 02 00 00 00 00 00 08"
cmd_des_dec = "02 03 00 00 00 00 00 08"
key_des = "12 34 56 78 90 ab cd ef"
des_data_len = 8

def init(c):        
    
    cracker.nut_set_voltage(3300) # 设置NUT工作电压
    cracker.nut_set_enable(1) # 使能NUT供电
    time.sleep(0.2)
    
    cracker.osc_set_analog_gain(1, 60) # 设置模拟信号增益       
    cracker.osc_set_sample_len(sample_length) # 设置采样长度

    cracker.osc_set_analog_trigger_source(2)
    cracker.osc_set_trigger_edge_level(0)

    if alg is AlgType.AES_ENC:
        cmd = cmd_set_aes_enc_key + key_aes
    elif alg is AlgType.AES_DEC:
        cmd = cmd_set_aes_dec_key + key_aes
    elif alg is AlgType.DES_ENC:
        cmd = cmd_set_des_enc_key + key_des
    else:
        cmd = cmd_set_des_dec_key + key_des

    cmd = cmd.replace(' ', '')
    cmd = bytes.fromhex(cmd)
    
    cracker.cracker_serial_data(6, cmd)
    # cracker.spi_transmit(cmd)
    # res = cracker.spi_receive(6)
    # print(res)

def do(c):
    # cracker.osc_force()
    # print(cracker.osc_is_triggered())
    
    if alg is AlgType.AES_ENC or alg is AlgType.AES_DEC:
        data = random.randbytes(aes_data_len)
    else:
        data = random.randbytes(des_data_len)
    
    if alg is AlgType.AES_ENC:
        cmd = cmd_aes_enc
    elif alg is AlgType.AES_DEC:
        cmd = cmd_aes_dec
    elif alg is AlgType.DES_ENC:
        cmd = cmd_des_enc
    else:
        cmd = cmd_des_dec
    
    cmd = cmd.replace(' ', '')
    cmd = bytes.fromhex(cmd)

    d = cmd + data
    # print(d.hex())

    if alg is AlgType.AES_ENC or alg is AlgType.AES_DEC:
        _, ret = cracker.cracker_serial_data(6+6 + aes_data_len, d)
        # ret = cracker.spi_receive(12 + aes_data_len)
        
        # print(len(ret))
        # print(ret.hex())
        return data + ret[-aes_data_len:]
    else:
        _, ret = cracker.cracker_serial_data(6 + des_data_len, d)
        # cracker.spi_transmit(d)
        # ret = cracker.spi_receive(6 + des_data_len)
        # print(len(ret))
        # print(ret.hex())
        # print(ret[-des_data_len:].hex())
        return data + ret[-des_data_len:]
               

acq = cn.new_acquisition(cracker, do=do, init= init)

In [ ]:
cn.panel(acq)

In [ ]:
cracker.connect()

In [ ]:
from cracknuts import logger

logger.set_level('DEBUG')

In [ ]:
cracker.osc_get_analog_wave(2, 0, 1024)

In [ ]:
cracker.osc_set_analog_trigger_source(1)
# cracker.osc_set_trigger_edge_level(0)